In [1]:
# Initialize
!git clone --recursive https://github.com/mitosagi/puzzdra-nnsolver
%cd puzzdra-nnsolver
!pip install --log=pip_log -e .

Cloning into 'puzzdra-nnsolver'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 287 (delta 24), reused 0 (delta 0), pack-reused 245
Receiving objects: 100% (287/287), 8.30 MiB | 14.18 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Submodule 'extern/pybind11' (https://github.com/pybind/pybind11) registered for path 'extern/pybind11'
Cloning into '/kaggle/working/puzzdra-nnsolver/extern/pybind11'...
remote: Enumerating objects: 27236, done.        
remote: Counting objects: 100% (8/8), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 27236 (delta 1), reused 4 (delta 1), pack-reused 27228        
Receiving objects: 100% (27236/27236), 10.31 MiB | 21.94 MiB/s, done.
Resolving deltas: 100% (19260/19260), done.
Submodule path 'extern/pybind11': checked out '8de7772cc72daca8e947b79b83fea46214931604'
/kaggle/working/puzzdra-nnsolver
Obtaining file:///k

In [42]:
import random
import numpy as np
from puzzpy import PuzzTable

drop_color = 3
board_width = 6
board_height = 5

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'

def npUint8(array):
    return np.array(array, dtype=np.uint8)

class PuzzBoard():        
    def reset(self):
        self.prev_action = 255
        
        while True:
            self.table = PuzzTable("".join([str(random.randrange(drop_color)) for i in range(board_width*board_height)]), random.randrange(board_width), random.randrange(board_height), 50) # n色陣　操作時間m秒
            if self.table.eval_otoshi() == 0:
                break

        return npUint8(self.table.get_table())
    
    def step(self):
        next_tables = self.table.next_tables()
        valid_actions = [action for action, table in enumerate(next_tables) if table.get_table()[0][0] != 127 and abs(action - self.prev_action) != 2]
        self.prev_action = random.choice(valid_actions)
        self.table = next_tables[self.prev_action]
        
        return self.prev_action, npUint8([(npUint8(table.get_table()) if action in valid_actions else np.zeros_like(npUint8(table.get_table()))) for action, table in enumerate(next_tables)])
    def render(self):
        tcolor = [bcolors.RED, bcolors.BLUE, bcolors.GREEN, bcolors.MAGENTA, bcolors.YELLOW, bcolors.BLACK]
        start = self.table.get_XY_as_table()
        table = self.table.get_table()
        for i in range(board_height):
            for j in range(board_width):
                if start[i][j] == 1:
                    print(tcolor[table[i][j]-1]  +  bcolors.UNDERLINE + "●" + bcolors.ENDC, end='')
                else:
                    print(tcolor[table[i][j]-1]  + "●" + bcolors.ENDC, end='')
            print('')

In [53]:
env = PuzzBoard()
obs = env.reset()
env.render()

n_steps = 3

for step in range(n_steps):
    print("Step {}".format(step + 1))
    action, obs = env.step()
    with np.printoptions(threshold=np.inf):
        print("action: ", action)
#         print(obs)
        env.render()

●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 1
action:  0
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 2
action:  1
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●
Step 3
action:  0
●●●●●●
●●●●●●
●●●●●●
●●●●●●
●●●●●●


In [59]:
# %%timeit # num_step=50のとき1_000サンプル生成に3.41sかかる。目標の100_000サンプル生成は６分弱。
def make_data(num_step):
    env = PuzzBoard()
    start = env.reset()

    sample_step = random.randrange(1, num_step + 1)

    for i in range(sample_step):
        answer, obs = env.step()
    sample_obs = obs

    for i in range(num_step - sample_step):
        answer, obs = env.step()
    end = obs[answer]
        
    return np.concatenate([npUint8([start]),sample_obs,npUint8([end])]), answer

from torch.utils.data import Dataset

class PuzzDataSet(Dataset):
    def __init__(self):
        samples = [make_data(50) for i in range(10_000)]
        self.data = npUint8([sample[0] for sample in samples])
        self.labels = npUint8([sample[1] for sample in samples])
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

%time dataset = PuzzDataSet()

CPU times: user 34.4 s, sys: 24.8 ms, total: 34.4 s
Wall time: 34.4 s


In [68]:
dataset.__len__()

10000